# Faza 3 uloha 1 (OneR)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
labor_train = pd.read_csv('train_preprocessed.csv', sep='\t', index_col=0)
labor_test = pd.read_csv('test_preprocessed.csv', sep='\t', index_col=0)

In [3]:
labor_train.head()

,indicator,er-cv,leukocyty,trombocyty,hemoglobin,ast,hematokrit
0,1.0,0.166667,0.401497,0.551966,0.742755,0.111111,0.322113
1,0.0,0.405000,0.525788,0.850466,0.096399,0.467167,0.767291
2,1.0,0.166667,0.812344,0.127427,0.278961,0.111111,0.213721
3,0.0,0.698214,0.787698,0.455711,0.888889,0.422187,0.512109
4,0.0,0.408464,0.944378,0.369594,0.128604,0.111111,0.255717


In [4]:
labor_test.describe()

,indicator,er-cv,leukocyty,trombocyty,hemoglobin,ast,hematokrit
count,2472.000000,2472.000000,2472.000000,2472.000000,2472.000000,2472.000000,2472.000000
mean,0.634304,0.493736,0.502336,0.499681,0.498993,0.500286,0.504455
std,0.481722,0.280514,0.282626,0.283822,0.287940,0.284910,0.283972
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.166667,0.246513,0.253332,0.251220,0.252247,0.251754
50%,1.000000,0.501564,0.508227,0.501003,0.500447,0.500071,0.502172
75%,1.000000,0.833333,0.748037,0.749898,0.744506,0.750488,0.742400
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


1. Leukocyty
2. Hematokrit
3. Hemoglobin
4. Er-cv
5. Trombocyty
6. Ast

In [5]:
def one_r(df, atributes):
    """
    OneR algoritmus dostane trenovaci a testovaci dataset a atributy
    postupne prechadza atributy a rozdeli mnozinu (0; 1) na intervaly po 0.1
    pre kazdu tuto triedu zrata kolko indikatorov je 1 a kolko 0, pre 1 prirata
    a pre 0 odrata jednotu od hodnoty v poli arr[] (pole arr reprezentuje hodnotu pre kazdu triedu daneho atributu)
    
    Nasledne sa prechadza znovu trenovaci dataset, pre kazdu hodnotu sa priradi indicator a porovna z uz danym indikatorom
    Takto sa pre kazdy atribut vyrata hodnota accuracy, a na konci sa zoberie atribut s najvacsou accuracy
    a pole hodnot pre kazdu triedu tohoto atributu
    """
    arr_accuracy = [0] * len(atributes)
    arr_all = []
    for index, atribute in enumerate(atributes):
        arr = [0]*10
        accuracy = 0
        for i, row in df.iterrows():
            for j in range(10):
                if j/10 <= row[atribute] < (j+1)/10:
                    if row["indicator"] == 1:
                        arr[j] += 1
                    else:
                        arr[j] -= 1
        for i, row in df.iterrows():
            for j in range(10):
                if j/10 <= row[atribute] < (j+1)/10:
                    if arr[j] > 0:
                        if row['indicator'] == 1:
                            accuracy += 1
                        else:
                            accuracy -= 1
                    else:
                        if row['indicator'] == 0:
                            accuracy += 1
                        else:
                            accuracy -= 1
        print(arr)
        print(f"accuracy: {accuracy}")
        arr_all.append(arr.copy())
        arr_accuracy[index] = accuracy
        print()
    return (atributes[arr_accuracy.index(max(arr_accuracy))], arr_all[arr_accuracy.index(max(arr_accuracy))])


def use_one_r(new, target_atribute, result):
    """
    Use OneR dostane nove data, ktore maju byt klasifikovane, atribut s najvacsou accuracy a result (pole hodnot atributu)
    Najde triedu do ktorej patri nova hodnota (v target_atribut) a vrati hodnotu jeho indicatora (0 alebo 1)
    """
    for j in range(10):
        if j/10 <= new[target_atribute] < (j+1)/10:
            if result[j] > 0:
                return 1
            else:
                return 0
    return 1
            
# tymto sposobom mi staci jeden krat natrenovat algoritmus, a mozem ho potom rychlo vela krat pouzit

In [6]:
target_atribute, target_arr = one_r(labor_train, ['leukocyty', 'hematokrit', 'hemoglobin', 'er-cv', 'trombocyty', 'ast'])

[490, 753, 661, 385, 2, -63, -40, -38, -44, 55]
accuracy: 2531

[237, 1040, 406, 283, 110, 35, 1, 12, -10, 47]
accuracy: 2181

[355, 396, 378, 375, 383, 323, 273, 196, -427, -91]
accuracy: 3197

[139, 468, 77, 194, 194, 260, 178, 99, 457, 97]
accuracy: 2163

[235, 255, 210, 176, 238, 212, 238, 238, 183, 178]
accuracy: 2163

[0, 407, 168, 217, 238, 243, 225, 182, 451, 30]
accuracy: 2161



In [7]:
"""
V cykle prechadzam testovaci dataset a pustam ho do funkcie use_one_r po riadkoch
Na konci zratam, kolko hodnot bolo priradenych spravne
"""
y_true = []
y_pred = []
for i, row in labor_test.iterrows():
    if use_one_r(row, target_atribute, target_arr) == None:
        print("now")
        use_one_r(row, target_atribute, target_arr)
    y_pred.append(use_one_r(row, target_atribute, target_arr))
    y_true.append(int(row['indicator']))

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print("Accuracy = ", accuracy_score(y_true, y_pred))
print("Precision = ", precision_score(y_true, y_pred))
print("Recall = ", recall_score(y_true, y_pred))

Accuracy =  0.7148058252427184
Precision =  0.7211686314710405
Recall =  0.8973214285714286
